<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-02-15 16:05:46--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-02-15 16:05:47--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  28.7MB/s    in 57s     

2024-02-15 16:06:44 (24.5 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [5]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [7]:
# method most_simmilar
# Находим топ-5 слов, близких к слову 'dog'
top_5_similar = wv_embeddings.similar_by_word('dog', topn=5)

top_5_similar

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [9]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)

tokenizer = MyTokenizer()

In [8]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
    question: строка
    embeddings: наше векторное представление
    dim: размер любого вектора в нашем представлении

    Преобразует вопрос в векторное представление, используя среднее векторов всех слов.
    """
    tokens = tokenizer.tokenize(question)
    vectors = []

    for token in tokens:
        if token in embeddings:
            vectors.append(embeddings[token])

    if vectors:
        question_vec = np.mean(vectors, axis=0)
    else:
        question_vec = np.zeros(dim)

    return question_vec

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [14]:
question = "I love neural networks"
# Использование функции question_to_vec для получения вектора предложения
question_vec = question_to_vec(question, wv_embeddings, tokenizer)

# Получение третьей компоненты вектора и округление до двух знаков после запятой
third_component = round(question_vec[2], 2)

print(third_component)

-1.29


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [15]:
def hits_count(dup_ranks, k):
    # Считаем количество дубликатов, которые находятся на позиции k или выше
    hits_value = sum(1 for rank in dup_ranks if rank <= k)
    return hits_value / len(dup_ranks) if dup_ranks else 0


In [16]:
def dcg_score(dup_ranks, k):
    dcg_value = sum((1 / np.log2(rank + 1)) for rank in dup_ranks if rank <= k)
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [17]:
import pandas as pd

In [19]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [candidates.index(copy_answers[0]) + 1 for candidates in candidates_ranking]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [20]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
#!unzip stackoverflow_similar_questions.zip
!unzip "/content/drive/MyDrive/stackoverflow_similar_questions.zip"

Archive:  /content/drive/MyDrive/stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [30]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        components = line.strip().split('\t')
        data.append(components)
    return data

Нам понадобиться только файл validation.

In [31]:
validation_data = read_corpus('/content/data/validation.tsv')

Кол-во строк

In [32]:
len(validation_data)

3760

Размер нескольких первых строк

In [33]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [34]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [35]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_vec = question_to_vec(question, embeddings, tokenizer, dim)
    candidate_vecs = [question_to_vec(candidate, embeddings, tokenizer, dim) for candidate in candidates]

    # Вычисляем косинусное сходство между вектором вопроса и векторами кандидатов
    similarities = cosine_similarity([question_vec], candidate_vecs)[0]

    # Сортируем кандидатов на основе косинусного сходства
    ranked_candidates = sorted(enumerate(candidates), key=lambda x: similarities[x[0]], reverse=True)

    # Возвращаем список пар (начальная позиция, кандидат), отсортированный по убыванию сходства
    return [(pos, candidate) for pos, candidate in ranked_candidates]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [36]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [37]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [39]:
# должно вывести
results = [
    [(1, 'C# create cookie from string and send it'),
     (0, 'Convert Google results object (pure js) to Python object'),
     (2, 'How to use jQuery AJAX for an outside domain?')],

    [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
    (*, 'select2 not displaying search results'), #скрыт
    (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

#### Ответ 5:
* 0, 2, 1


Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [40]:
from tqdm.notebook import tqdm

In [42]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i >= max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    original_pos_of_duplicate = ex.index(ex[0])
    ranked_pos_of_duplicate = [r[1] for r in ranks].index(ex[0]) + 1
    wv_ranking.append(ranked_pos_of_duplicate)

'''
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index('0') + 1)
    '''

  0%|          | 0/3760 [00:00<?, ?it/s]

"\nfor i, line in enumerate(tqdm(validation_data)):\n    if i == max_validation_examples:\n        break\n    q, *ex = line\n    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)\n    wv_ranking.append([r[0] for r in ranks].index('0') + 1)\n    "

In [43]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 285.000 | Hits@   1: 0.285
DCG@   5: 341.622 | Hits@   5: 0.393
DCG@  10: 359.693 | Hits@  10: 0.449
DCG@ 100: 405.681 | Hits@ 100: 0.679
DCG@ 500: 431.275 | Hits@ 500: 0.879
DCG@1000: 443.923 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [47]:
train_data = read_corpus('/content/data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

При обучении модели Word2Vec одним из ключевых параметров, влияющих на качество векторных представлений слов, является размер окна (window). Размер окна определяет максимальное расстояние между текущим и предсказываемым словом в предложении. В контексте склеивания вопросов в пары и обучения на них, выбор размера окна становится особенно важным, поскольку он влияет на то, как модель учится улавливать связи между словами в рамках одного вопроса и между двумя разными вопросами.

#### Выбор размера окна
* Маленькое окно (например, 2-5): Будет полезно, если ваша основная цель — уловить синтаксические связи между словами. Это подходит для ситуаций, когда важны ближайшие контексты слов, но не лучший выбор для захвата более широких семантических связей между частями вопросов.

* Большое окно (например, 10-20): Подходит для захвата более широких семантических связей между словами, что может быть полезно при склеивании вопросов в пары, так как это помогает модели лучше понимать контекстуальные связи между различными частями сложных вопросов или между двумя разными вопросами.

In [48]:
window = 12

In [57]:
words = [tokenizer.tokenize(line[0]) for line in train_data]

In [58]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=window).wv

In [64]:
wv_ranking = []
max_train_examples = 1000000
for i, line in enumerate(tqdm(train_data)):
    if i >= max_train_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    original_pos_of_duplicate = ex.index(ex[0])
    ranked_pos_of_duplicate = [r[1] for r in ranks].index(ex[0]) + 1
    wv_ranking.append(ranked_pos_of_duplicate)

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [65]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 902317.000 | Hits@   1: 0.902
DCG@   5: 961408.392 | Hits@   5: 1.000
DCG@  10: 961532.763 | Hits@  10: 1.000
DCG@ 100: 961540.397 | Hits@ 100: 1.000
DCG@ 500: 961540.397 | Hits@ 500: 1.000
DCG@1000: 961540.397 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


#### Какой принцип токенизации даёт качество лучше и почему?
* Простая токенизация (разбиение по пробелам или на слова с удалением пунктуации) часто работает хорошо для языков с чётким разделением слов пробелами, но может упускать нюансы, связанные с сокращениями, составными словами или спецификой языка.
* Продвинутая токенизация (использование библиотек типа NLTK, SpaCy, или BERT токенизаторов), которая учитывает контекст слов и может корректно обрабатывать сокращения, сложные слова, а также предоставлять лемматизацию или стемминг, часто даёт лучшие результаты на сложных текстовых данных.

#### Помогает ли нормализация слов?
* Нормализация слов, включая лемматизацию и стемминг, может существенно улучшить качество моделей NLP, поскольку она уменьшает вариативность форм слов и сокращает размер словаря, делая данные более однородными и упрощая задачу для модели.

#### Какие эмбеддинги лучше справляются с задачей и почему?
* Предобученные эмбеддинги (например, Word2Vec, GloVe, FastText) часто предоставляют хорошую основу за счёт использования больших корпусов данных и могут быть эффективны в задачах, где тексты схожи по стилю и тематике с текстами обучающего корпуса.
* Специализированные эмбеддинги, обученные непосредственно на данных конкретной задачи, могут дать лучшие результаты в случае, если имеется достаточно большой и релевантный датасет, поскольку они лучше учитывают специфику задачи.

#### Почему получилось плохое качество решения задачи?
Плохое качество решения может быть обусловлено несколькими факторами:

* Недостаточное количество или качество данных для обучения.
* Неподходящая предобработка данных, включая выбор метода токенизации и нормализации.
* Неоптимальный выбор параметров модели или самой модели для задачи.
* Несоответствие между предобученными эмбеддингами и доменом задачи.

#### Предложите свой подход к решению задачи.
Для улучшения качества модели можно предложить следующие шаги:

* Эксперименты с предобработкой: пробовать разные методы токенизации и нормализации, а также очистку данных от шума.
* Использование контекстуальных эмбеддингов, таких как BERT, GPT или их адаптации под конкретную задачу, которые могут лучше учитывать семантику текста.
* Обогащение данных: использование дополнительных источников данных для увеличения объёма и разнообразия обучающего датасета.
* Файн-тюнинг модели: подбор гиперпараметров и архитектуры модели для максимальной эффективности на данных задачи.